<center>Національний університет "Львівська політехніка"</center>
<center>Кафедра інформаційних систем та мереж</center>

## <center> Екзаменаційна робота</center>
### <center> з дисципліни "Глибинне навчання" </center>

студента групи СА-43

Мозоля Назарія

<center> Варіант 12</center>
<center> 30.05.2024 </center>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### утиліти

In [3]:
def _load_t31():
    from PIL.Image import open
    from urllib.request import urlopen
    print("https://www.kaggle.com/datasets/cashbowman/ai-generated-images-vs-real-images")
    _, axs = plt.subplots(1, 2, figsize=(10, 5))

    gen_im = np.array(open(urlopen("https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcTkh7eZ6a5hdZXRlOqz_fD2rtb8mnDYYn7k7I7z0XeJvYKZ30ZO")))
    real_im = np.array(open(urlopen("https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcR_JVJI8gLgFrT1A3uC_iVscHDwGhdMLwF8I5RI4CxedXmVKeDs")))
    for i, im, title in zip(range(2), [gen_im, real_im], ['Generated image', 'Real image']):
        axs[i].imshow(im)
        axs[i].set_title(f"{title} [size: {im.shape}]")

def _load_t32():
    from IPython.display import display
    print("https://www.kaggle.com/datasets/prajwaldongre/llm-detect-ai-generated-vs-student-generated-text")
    df = pd.read_csv('https://drive.google.com/uc?export=download&id='+"https://drive.google.com/file/d/1YeGjb7P6DquMZir8pYxFNiYVaCCATzdV/view?usp=drive_link".split('/')[-2])
    pd.options.display.max_colwidth=300
    display(df[:4])
    return df

def _load_t33():
    from IPython.display import display
    print("https://www.kaggle.com/datasets/bhavikbb/password-strength-classifier-dataset")
    df = pd.read_csv(
        'https://drive.google.com/uc?export=download&id='+"https://drive.google.com/file/d/10tEF3BO2eb6uyTs0m_bQcWtL-rMxGKEb/view?usp=drive_link".split('/')[-2],
        on_bad_lines='skip',
        nrows=10000
        )
    display(df[:13])
    return df

def _load_t34():
    print("https://www.kaggle.com/datasets/equilibriumm/sleep-efficiency")
    df = pd.read_csv(
        'https://drive.google.com/uc?export=download&id='+"https://drive.google.com/file/d/15GqxECYVNcbxJCATTbgHcpInhVt9zF0F/view?usp=drive_link".split('/')[-2],
        on_bad_lines='skip',
        nrows=10000
        )
    display(df[:10])
    return df


def load_task3_data(variant: int) -> None | pd.DataFrame:
    print('Kaggle page:', end=' ')
    return [_load_t31, _load_t32, _load_t33, _load_t34][variant % 4 - 1]()

## Завдання 1

```
Дайте відповідь на питання у (майже) довільній формі.
```

Призначення та принцип роботи шарів пулінгу/об’єднання (pooling) в CNN.

Polling -- механізм зменшення розмірності вхідних даних. Призначений для зменшення обчислювальгого ресурсу моделі(пришвидшення роботи через меншу розмірність вхідних даних) та запобіганню overfit`у.



Представляє з себе ковзаючий кернел який пересувається по вхідній матриці виконуючи певну агреговану функцію над даними які перекриває в ориг.матриці.

Має два тренувлаьні параметри: розмір ядра(зазвичай однаковий в кожному вимірі і є непарними числом) та страйд(кількість клітинок вправо чи вниз на які переміщається ядро за один хід).

Має дві агреговані функції: maxpool-- витягання найбільшого елементу з перекритої матриці, та avgpool-- витягання середнього значення з усіх перекритих елементів матриці. Також існує протилежна функція до пулінгу-- upscale, використовується наприклад в unet.

Використання:
-є звичайним шаром в моделі
-зазвичай розміщається після кількох ковуляційних шарів(наприклад як в vgg16), або на початку та кінці feature extraction частини(як в resnet).
-конвуляціний шар(шари) разом з шаром пулінгу утворюють конвуляціний блок.
-формула зменшення розмірності аналогічна до конвулюції(проте має тільки kernel_size та padding): w_out=(w_in+2padd-kernel_size)/stride+1





## Завдання 2

```
Дайте відповідь на питання у (майже) довільній формі.
```

Що таке затухання градієнта? Як skip-connection дозволяє боротися із затуханням?

Затухання градієнта-- явище характерне для глибоких мереж(без використання skip-con), явище під час backpropagation, коли найдільші шари від виходу мережі слабооновлюють параметри або взагалі не оноволюють їх через те що градієнт для них близький до нуля. Також характерне для моделей які мають сигмоїду чи тангсенс гіперболічний(чи схоже) як фунції активації, бо під час backprop. стискають вхідні значення до дуже малих цифр, відповідно і градієнт теж дуже малий, оновлення параметрів теж.

Skip-connection- механізм який був представлений в моделі ResNet.
Вплив:
-Його можна розуміти як прямий шлях для градієнту який рухається від кінця мережі до початку, оминаючи шари чи шари.
-Зберігає початкові дані, бо власне скіп-кон. дозволяє оминати шари, тому після проходження даних, кожен раз ми маємо на виході skip-con оригінальні дані плюс якийсь feature extraction(результат роботи шару) який накладений на них.

Формула:
H_out(x)=f(x)+x-- вихід з skip-con це власний результат роботи шару(f(x)), який накладений на оригінальні дані(x).

backprop:
dH_out(x)/dx=d(F(x))/dx+d(x)/dx=f`+1. Відповідно маємо відсутність занулення градієнту бо завжди додаємо одиницю. Саме це і дозволяє обходити проблему затухання.
В ResNet скіп-кон. охоплював по два шари і називався блоком. Кожен блок відповідав за кожен клас зображення.

## Задача 3

```
Запропонуйте рішення проблеми/завдання, використовуючи моделі ГН (як мінімум: пайплайн препроцесингу + архітектура моделі + гіперпараметри для навчання або готовий пайплайн рішення на базі наперед навчених моделей)
```

In [4]:
df = load_task3_data(12)

Kaggle page: https://www.kaggle.com/datasets/equilibriumm/sleep-efficiency


,ID,Age,Gender,Bedtime,Wakeup time,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Smoking status,Exercise frequency
0,1,65,Female,2021-03-06 01:00:00,2021-03-06 07:00:00,6.0,0.88,18,70,12,0.0,0.0,0.0,Yes,3.0
1,2,69,Male,2021-12-05 02:00:00,2021-12-05 09:00:00,7.0,0.66,19,28,53,3.0,0.0,3.0,Yes,3.0
2,3,40,Female,2021-05-25 21:30:00,2021-05-25 05:30:00,8.0,0.89,20,70,10,1.0,0.0,0.0,No,3.0
3,4,40,Female,2021-11-03 02:30:00,2021-11-03 08:30:00,6.0,0.51,23,25,52,3.0,50.0,5.0,Yes,1.0
4,5,57,Male,2021-03-13 01:00:00,2021-03-13 09:00:00,8.0,0.76,27,55,18,3.0,0.0,3.0,No,3.0
5,6,36,Female,2021-07-01 21:00:00,2021-07-01 04:30:00,7.5,0.90,23,60,17,0.0,NaN,0.0,No,1.0
6,7,27,Female,2021-07-21 21:00:00,2021-07-21 03:00:00,6.0,0.54,28,25,47,2.0,50.0,0.0,Yes,1.0
7,8,53,Male,2021-08-16 00:30:00,2021-08-16 10:30:00,10.0,0.90,28,52,20,0.0,50.0,0.0,Yes,3.0
8,9,41,Female,2021-04-05 02:30:00,2021-04-05 08:30:00,6.0,0.79,28,55,17,3.0,50.0,0.0,No,1.0
9,10,11,Female,2021-09-16 01:00:00,2021-09-16 10:00:00,9.0,0.55,18,37,45,4.0,0.0,0.0,No,0.0


Задача: прогноз якості сну-- регресія. В кінці моделі relu, 3 dense, по 150. Лосс-- MAE
Обробка:
1. drop('id')
2. Стандартизацію для віку, бо точно має близький до нормального розподілу
3. Стать- one-hot
4. Статус куріння one-hot
5. datetime колонки(badtime, wakeuptime)-- виділення компонент, додача циклічних зміних, лаг, козваюче середнє.
6. всі персентежі -- нормалізація.

## Задача 4

```
Для заданої задачі розробити шаблон(-и) промптів до LLM для її розв'язання
```

Ваш одногрупник/одногрупниця розробиляють Python pair-програмера бота, і ви хочете допомогти йому/їй/їм з оцінкою якості генерованого коду, тобто з валідацією результатів генерування. У вас доступний невеликий датасет з коментарем, очікуваним кодом та реально згенерованим кодом моделлю.


System

"Встановлення temp=0.7, topk=0.9"

In [ ]:
#Інструкція:
Ти оцінюєш якість коду, згенерованого користувачем для створення Python pair-програмера ботом. Твоє завдання — порівняти згенерований код з очікуваним кодом, базуючись на наданому коментарі. Використовуй критерії точності, ефективності та відповідності коментарю для оцінки

#Контекст:
Користувач розробляє бота для pair-програмування на Python, і тобі потрібно допомогти йому з валідацією результатів. У тебе є невеликий датасет, який містить коментар, очікуваний код та код згенерований моделлю. Твоʼ завдання — оцінити, наскільки згенерований код відповідає очікуваному та коментарю.

#Приклад:
Вхідні дані:
-Коментар: "Написати функцію, яка приймає два числа і повертає їх суму."
-Очікуваний код:
python```
def add_numbers(a, b):
    return a + b
```
-Згенерований код:
python```
def add_numbers(x, y):
    return x + y
```


#Вихідні дані:
-Оцінка: 9
-Коментар до оцінки: Код майже ідентичний очікуваному. Відмінності лише в назвах параметрів, що не впливає на функціональність.

#Обмеження та критерії генерування:
-Максимальна довжина відповіді: 500 символів.
-Код має бути читабельним та виконуватись без помилок.
-Оцінка має бути обʼєктивною та базуватись на відповідності коментарю та очікуваному коду.

User

In [ ]:
Comment: "Write a function that multiplies two numbers."
Expected Code:

python```
def multiply_numbers(a, b):
    return a * b
```
Generated code:
python```
def multiply_numbers(x, y):
    return x * y
```


Asistant

In [ ]:
Score:10
Comment:'The generated code perfectly matches the expected code in terms of functionality. The only difference is the parameter names, which do not impact the operation of the function'